# Data Preparation

In [1]:
import kaggle
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
# kaggle.api.dataset_download_files("mlg-ulb/creditcardfraud", path="../data/raw", unzip=True)
df_full = pd.read_csv("../data/raw/creditcard.csv")

Z-Noramlisation

In [20]:
df_normalised = df_full
for column in df_normalised.columns[:-1]:
    std = np.std(df_normalised[column])
    mean = np.mean(df_normalised[column])
    df_normalised[column] = df_normalised[column] - mean
    df_normalised[column] = df_normalised[column] / std

df_normalised.to_csv("../data/interim/creditcard_z_normalised.csv", index=False)

## Min Max Scaling

In [4]:
scaler = MinMaxScaler()
scaler.fit(df_full)
scaled_data = scaler.transform(df_full)
scaled_df = pd.DataFrame(scaled_data, columns=df_full.columns)
scaled_df.to_csv("../data/interim/creditcard_min_max.csv", index=False)

# Outlier Elimination

In [2]:
df_scaled = pd.read_csv("../data/interim/creditcard_min_max.csv")

In [28]:
def remove_outliers(df, end_column=30):
    # Select columns up to end_column
    df_selected = df.iloc[:, :end_column]

    # Calculate mean and standard deviation for each selected column
    mean = df_selected.mean()
    std = df_selected.std()

    # Define threshold for outliers
    threshold = 3 * std

    # Create a boolean mask for outliers
    outliers = np.abs(df_selected - mean) > threshold

    # Return DataFrame without outliers
    return df[~outliers.any(axis=1)]


# Remove outliers up to column 30
cleaned_df = remove_outliers(df_scaled)

In [2]:
cleaned_df.to_csv("../data/interim/outliers_removed.csv", index=False)

NameError: name 'cleaned_df' is not defined

# First X Percent

In [2]:
# Load the DataFrame from the CSV file
df_min_max = pd.read_csv("../data/interim/creditcard_min_max.csv")
df_min_max_array = df_min_max.values
total_length = len(df_min_max_array)

# Calculate 20% of the total length
twenty_percent_length = int(0.1 * total_length)

# Slice the first 20 percent of the values in df_min_max_array
df_min_max_array_cut = df_min_max_array[:twenty_percent_length]

# Create a DataFrame from the sliced array and specify columns
df_min_max_cut = pd.DataFrame(df_min_max_array_cut, columns=df_min_max.columns)

# Save the sliced data to a new CSV file
df_min_max_cut.to_csv("../data/interim/cut_data.csv", index=False)



# Splitting the Data in Train Test Split

In [2]:

df_credit = pd.read_csv("../data/interim/creditcard_min_max.csv")
credit_array = df_credit.values


# Specify the test_size and validation_size according to your needs
test_size = 0.2  # 80% training, 20% test
validation_size = 0.1  # 80% training, 10% validation

# Split into training and temporary data
train_data, temp_data = train_test_split(
    df_credit, test_size=test_size + validation_size, random_state=42
)

# Split the temporary data into test and validation
test_data, validation_data = train_test_split(
    temp_data,
    test_size=validation_size / (test_size + validation_size),
    random_state=69)

In [3]:
# save the unsorted
train_data.to_csv("../data/processed/shuffled_train_data_mm.csv", index=False)
test_data.to_csv("../data/processed/shuffled_test_data_mm.csv", index=False)
validation_data.to_csv("../data/processed/shuffled_val_data_mm.csv", index=False)

In [55]:
sorted_train_data = train_data.sort_values(by="Time")
sorted_test_data = test_data.sort_values(by="Time")
sorted_val_data = validation_data.sort_values(by="Time")

In [56]:
print('test_data', sum(test_data.Class) / len(test_data))
print("train_data", sum(train_data.Class) / len(train_data))
print("val_data", sum(validation_data.Class) / len(validation_data))

test_data 0.0016151118289385905
train_data 0.0017856784574948336
val_data 0.001544889575506478


In [58]:
sorted_train_data.to_csv("../data/processed/sorted_train_data_mm.csv", index=False)
sorted_test_data.to_csv("../data/processed/sorted_test_data_mm.csv", index=False)
sorted_val_data.to_csv("../data/processed/sorted_val_data_mm.csv", index=False)

# Random Sampling out of the whole dataframe

In [13]:
df_full = pd.read_csv("../data/interim/creditcard_min_max.csv")

total_rows = len(df_full)
subset_size = int(0.1 * total_rows)

# Generate random indices for the first subset and sample
df_train = df_full.sample(n=subset_size, replace=False)
df_train_incides = df_full.sample(n=subset_size, replace=False).index

# drop the indices to be sure no overlapping
df_dropped = df_full.drop(df_train_incides)
df_test = df_full.sample(n=subset_size, replace=False)

len(df_train) == len(df_test)

# save the random sample df

True

In [14]:
df_train.to_csv("../data/interim/privacy_testing/train_random_sample.csv", index=False)
df_test.to_csv("../data/interim/privacy_testing/test_random_sample.csv", index=False)

# Oversampled Minority Class

In [3]:
df_train = pd.read_csv("../data/processed/sorted_train_data_mm.csv")

In [2]:
from imblearn.over_sampling import RandomOverSampler

In [45]:
X_train = df_train.drop(
    columns=["Class"]
) 
y_train = df_train["Class"]

# Random oversampling with a sampling strategy of 10 percent
ros = RandomOverSampler(sampling_strategy=0.1, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

df_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_resampled["Class"] = y_train_resampled
df_resampled.sort_values("Time", inplace=True)

In [47]:
df_resampled.to_csv("../data/processed/train_oversampled_01.csv", index=False)

In [48]:
ros = RandomOverSampler(sampling_strategy=0.2, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Random oversampling with a sampling strategy of 20 percent
df_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_resampled["Class"] = y_train_resampled
df_resampled.sort_values("Time", inplace=True)

df_resampled.to_csv("../data/processed/train_oversampled_02.csv", index=False)

In [50]:
X_train = df_train.drop(columns=["Class"])
y_train = df_train["Class"]

# Random oversampling with a sampling strategy of 5 percent
ros = RandomOverSampler(sampling_strategy=0.05, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

df_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_resampled["Class"] = y_train_resampled
df_resampled.sort_values("Time", inplace=True)
df_resampled.to_csv("../data/processed/train_oversampled_005.csv", index=False)

In [53]:
X_train = df_train.drop(columns=["Class"])
y_train = df_train["Class"]

# Random oversampling with a sampling strategy of 1 percent
ros = RandomOverSampler(sampling_strategy=0.01, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

df_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_resampled["Class"] = y_train_resampled
df_resampled.sort_values("Time", inplace=True)
df_resampled.to_csv("../data/processed/train_oversampled_001.csv", index=False)

In [6]:
df_train = pd.read_csv("../data/interim/cut_data.csv")

In [5]:
X_train = df_train.drop(columns=["Class"])
y_train = df_train["Class"]

# Random oversampling with a sampling strategy of 5 percent
ros = RandomOverSampler(sampling_strategy=0.05, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

df_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_resampled["Class"] = y_train_resampled
df_resampled.sort_values("Time", inplace=True)
df_resampled.to_csv("../data/interim/train_cut_oversampled_005.csv", index=False)